# StyleTTS2 Training on Google Colab
## Train Cigdem TTS Model

This notebook will help you train your Turkish TTS model on Google Colab with free GPU.

## Step 1: Check GPU and Setup Environment

In [ ]:
# Check GPU availability
!nvidia-smi
import torch
print(f"\nPyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

## Step 2: Clone StyleTTS2 Repository

In [ ]:
# Clone your repository
!git clone https://github.com/ElkhanAbbasov/TTS_Cigdem.git
%cd TTS_Cigdem

# Verify we're in the right place
!pwd
!ls -la

## Step 3: Install Dependencies

In [ ]:
# Install required packages (INCLUDING einops-exts and click)
!pip install -q SoundFile torchaudio munch pydub pyyaml librosa nltk matplotlib accelerate transformers einops einops-exts tqdm click
!pip install -q git+https://github.com/resemble-ai/monotonic_align.git

print("✅ All dependencies installed!")

## Step 4: Mount Google Drive (for saving checkpoints)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Create checkpoint backup folder in Drive
!mkdir -p '/content/drive/MyDrive/Cigdem_TTS_Checkpoints'
print("✅ Google Drive mounted!")

## Step 5: Verify Training Data

In [ ]:
# Check if training data exists
import os

train_list = "Data/my_train_list.txt"
if os.path.exists(train_list):
    with open(train_list, 'r', encoding='utf-8') as f:
        lines = f.readlines()
    print(f"✅ Training data found: {len(lines)} samples")
    print("\nFirst 3 samples:")
    for line in lines[:3]:
        print(f"  {line.strip()}")
    
    # Quick check that all audio files can be loaded
    print("\n🔍 Checking all audio files...")
    import soundfile as sf
    all_valid = True
    for i, line in enumerate(lines, 1):
        path = line.strip().split('|')[0]
        try:
            wave, sr = sf.read(path)
            print(f"  ✓ {i}: {path} ({sr} Hz)")
        except Exception as e:
            print(f"  ✗ {i}: {path} - ERROR: {e}")
            all_valid = False
    
    if all_valid:
        print("\n✅ All audio files can be loaded!")
    else:
        print("\n⚠️ Some audio files have issues!")
else:
    print("❌ Training data not found!")
    print("\n📂 Current directory:", os.getcwd())
    print("\n📋 Directory contents:")
    !ls -la
    print("\n📋 Looking for Data folder:")
    !ls -la Data/ 2>/dev/null || echo "Data folder not found"
    print("\n⚠️ Make sure the repository was cloned correctly!")

## Step 6: Update Config for Fresh Training

We need to modify the config to start fresh training (no pretrained checkpoint loading).

## Step 7: Start Training

**IMPORTANT:** 
- Free Colab sessions last ~12 hours max
- Training will run continuously
- Checkpoints saved every 2 epochs to Models/Cigdem_TTS/
- Backup to Google Drive regularly!
- If disconnected, re-run from Step 6 to resume from last checkpoint

In [ ]:
# Update config to disable pretrained model loading (start fresh)
import yaml

config_path = "Configs/config_ft.yml"
with open(config_path, 'r') as f:
    config = yaml.safe_load(f)

# Check if pretrained model exists, if not, disable it
pretrained = config.get('pretrained_model', '')
if pretrained and not os.path.exists(pretrained):
    print(f"⚠️  Pretrained model not found: {pretrained}")
    print("Setting second_stage_load_pretrained: false to start fresh training")
    config['second_stage_load_pretrained'] = False
    config['pretrained_model'] = ""
    
    # Save updated config
    with open(config_path, 'w') as f:
        yaml.dump(config, f, default_flow_style=False)
    print("✅ Config updated for fresh training!")
else:
    print(f"✅ Using pretrained model: {pretrained}")

# Display key config settings
print(f"\n📋 Training Configuration:")
print(f"  Batch size: {config.get('batch_size', 'N/A')}")
print(f"  Epochs: {config.get('epochs', 'N/A')}")
print(f"  Save frequency: {config.get('save_freq', 'N/A')} epochs")
print(f"  Log dir: {config.get('log_dir', 'N/A')}")
print(f"  Second stage pretrained: {config.get('second_stage_load_pretrained', 'N/A')}")

In [ ]:
# Start training - runs continuously until complete or stopped
# If this is a resume, it will automatically load the latest checkpoint
!python train_finetune.py --config_path Configs/config_ft.yml

## Step 8: Backup Checkpoints to Google Drive (Run periodically)

**Run this cell every 2-3 hours while training runs above!**

In [ ]:
# Copy checkpoints to Google Drive
!cp Models/Cigdem_TTS/epoch_*.pth '/content/drive/MyDrive/Cigdem_TTS_Checkpoints/' 2>/dev/null || echo "No checkpoints yet"
!cp Models/Cigdem_TTS/train.log '/content/drive/MyDrive/Cigdem_TTS_Checkpoints/' 2>/dev/null || echo "No log yet"

# List backed up files
!ls -lh '/content/drive/MyDrive/Cigdem_TTS_Checkpoints/'
print("\n✅ Checkpoints backed up!")

## Step 9: View Training Progress

In [ ]:
# Display last 20 lines of training log
!tail -n 20 Models/Cigdem_TTS/train.log

## Step 9: Download Results (After epoch 20+)

Download checkpoints to test locally

In [ ]:
# Zip checkpoints for download
!zip -r cigdem_tts_checkpoints.zip Models/Cigdem_TTS/epoch_*.pth Models/Cigdem_TTS/train.log

from google.colab import files
files.download('cigdem_tts_checkpoints.zip')

print("✅ Ready to download!")

---

## 📝 Important Notes:

### Training Process:
- Training runs **continuously** once started (Step 6)
- Goes from epoch 1 → 100 automatically
- Each epoch takes ~2-3 minutes
- **Don't re-run Step 6** unless training stops!

### When to Test:
- **Epoch 20+**: First test (may still be noisy)
- **Epoch 30+**: Better quality expected
- **Epoch 50+**: Good quality for your voice

### If Colab Disconnects:
1. Restore checkpoints from Google Drive:
   ```python
   !cp '/content/drive/MyDrive/Cigdem_TTS_Checkpoints/epoch_*.pth' Models/Cigdem_TTS/
   ```
2. Re-run Step 6 (training cell)
3. It will automatically resume from last checkpoint!